In [1]:
import pandas as pd
import json
import numpy as np
from collections import defaultdict
exact_result = {}
exact_runtime = defaultdict(list)
multi_table_runtime = defaultdict(list)
single_table_runtime = defaultdict(list)
multi_table_results = defaultdict(list)
single_table_results = defaultdict(list)
with open("all_results.jsonl") as f:
    for line in f:
        result = json.loads(line)
        query_id = result["query_id"]
        if "multi_table_sample_time" in result and result["multi_table_sample_time"] != -1:
            multi_table_runtime[query_id].append(result["multi_table_sample_time"] + result["pilot_time"])
            multi_table_results[query_id].append(result["multi_table_sample_result"])
        elif "single_table_sample_time" in result and result["single_table_sample_time"] != -1:
            single_table_runtime[query_id].append(result["single_table_sample_time"] + result["pilot_time"])
            single_table_results[query_id].append(result["single_table_sample_result"])
        elif "exact_time" in result:
            exact_runtime[query_id].append(result["exact_time"])
            exact_result[query_id] = result["exact_result"]
print("multi_time", multi_table_runtime)
print("single_time", single_table_runtime)
        # # find the column is not of string type
        # print(query_id)
        # if multi_result.empty:
        #     print("multi_result is empty")
        # else:
        #     non_string_columns = []
        #     for column in multi_result.columns:
        #         if single_result[column].dtype != "object":
        #             non_string_columns.append(column)
        #     error = np.abs(single_result[non_string_columns] - exact_result[non_string_columns]) / exact_result[non_string_columns]
        #     max_error = error.max().max()
        #     print("multi table sample error: ", max_error*100)
        # if single_result.empty:
        #     print("single_result is empty")
        # else:
        #     non_string_columns = []
        #     for column in single_result.columns:
        #         if single_result[column].dtype != "object":
        #             non_string_columns.append(column)
        #     error = np.abs(single_result[non_string_columns] - exact_result[non_string_columns]) / exact_result[non_string_columns]
        #     max_error = error.max().max()
        #     print("single table sample error: ",max_error*100)


multi_time defaultdict(<class 'list'>, {5: [864.9892723560333], 7: [465.71962308883667], 12: [250.76098251342773], 14: [80.3570020198822]})
single_time defaultdict(<class 'list'>, {5: [256.5050542354584], 7: [78.18216681480408], 9: [423.03892850875854], 12: [18.59325909614563], 14: [23.255903720855713], 19: [478.28358006477356]})


In [2]:
for query_id in exact_result:
    print("query id", query_id)
    exact_runtime_avg = np.mean(exact_runtime[query_id])
    exact_result_query = pd.DataFrame(exact_result[query_id])
    if query_id in multi_table_runtime:
        multi_runtime_avg = np.mean(multi_table_runtime[query_id])
        speedup = exact_runtime_avg / multi_runtime_avg
        print("multi table sample speedup: ", speedup)
        for result in multi_table_results[query_id]:
            result = pd.DataFrame(result)
            non_string_columns = []
            for column in exact_result_query:
                if result[column].dtype != "object":
                    non_string_columns.append(column)
            error = np.abs(result[non_string_columns] - exact_result_query[non_string_columns]) / exact_result_query[non_string_columns]
            max_error = error.max().max()
            print("multi table sample error: ", max_error*100)
    if query_id in single_table_runtime:
        single_runtime_avg = np.mean(single_table_runtime[query_id])
        speedup = exact_runtime_avg / single_runtime_avg
        print("single table sample speedup: ", speedup)
        for result in single_table_results[query_id]:
            result = pd.DataFrame(result)
            non_string_columns = []
            for column in exact_result_query:
                if result[column].dtype != "object":
                    non_string_columns.append(column)
            error = np.abs(result[non_string_columns] - exact_result_query[non_string_columns]) / exact_result_query[non_string_columns]
            max_error = error.max().max()
            print("single table sample error: ", max_error*100)


query id 5
multi table sample speedup:  0.9716514952497065
multi table sample error:  3.9459394025748518
single table sample speedup:  3.2766142654179014
single table sample error:  99.99013097132685
query id 7
multi table sample speedup:  1.1691195618118937
multi table sample error:  68.43529667531777
single table sample speedup:  6.964272593807434
single table sample error:  0.6841936752326343
query id 8
query id 9
single table sample speedup:  4.953519738427374
single table sample error:  4.034675847877086
query id 12
multi table sample speedup:  2.0436408429209627
multi table sample error:  2.492579097340011
single table sample speedup:  27.561891276051977
single table sample error:  2.415044908018909
query id 14
multi table sample speedup:  6.007107310565834
multi table sample error:  3.3555359130357663
single table sample speedup:  20.756584654067623
single table sample error:  0.04783208589565921
query id 19
single table sample speedup:  0.9964526573902365
single table sample er